<a href="https://colab.research.google.com/github/dakilaledesma/Atomized_NN_Experiments/blob/main/resolution_differences/GPU_ResNet50_600_Control.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%%time
! unzip -q drive/MyDrive/UNC/H2022/orchidaceae_train.zip -d /content/

CPU times: user 176 ms, sys: 24.7 ms, total: 201 ms
Wall time: 26 s


In [3]:
! git clone https://github.com/rwightman/pytorch-image-models.git

Cloning into 'pytorch-image-models'...
remote: Enumerating objects: 10776, done.
remote: Counting objects: 100% (303/303), done.
remote: Compressing objects: 100% (172/172), done.
remote: Total 10776 (delta 161), reused 223 (delta 123), pack-reused 10473
Receiving objects: 100% (10776/10776), 20.47 MiB | 9.57 MiB/s, done.
Resolving deltas: 100% (7877/7877), done.


In [4]:
import fileinput
import sys

def replacement(file, previousw, nextw):
   for line in fileinput.input(file, inplace=1):
       line = line.replace(previousw, nextw)
       sys.stdout.write(line)

file = "/content/pytorch-image-models/timm/utils/checkpoint_saver.py"
replacement(file, "if os.path.exists(last_save_path):", "# if os.path.exists(last_save_path):")
replacement(file, "os.unlink(last_save_path)  # required for Windows support.", "# os.unlink(last_save_path)  # required for Windows support.")
replacement(file, "os.link(last_save_path, save_path)", "# os.link(last_save_path, save_path)")
replacement(file, "os.unlink(best_save_path)", "# os.unlink(best_save_path)")
replacement(file, "os.link(last_save_path, best_save_path)", "# os.link(last_save_path, best_save_path)")
replacement(file, "if os.path.exists(best_save_path):", "# if os.path.exists(best_save_path):")


In [ ]:
! python -u -m torch.distributed.launch --nproc_per_node=1 --nnodes=1 --node_rank=0 ./pytorch-image-models/train.py orchidaceae_train --model resnet50 --opt adabelief --lr 0.001 --epochs 80 --decay-epochs 3 --cooldown-epochs 0 --weight-decay 1e-4 --sched cosine -b 32 --input-size 3 600 600 --num-classes=300 --amp --pretrained --output drive/MyDrive/UNC/H2022/output/

/usr/local/lib/python3.7/dist-packages/torch/distributed/launch.py:186: FutureWarning: The module torch.distributed.launch is deprecated
and will be removed in future. Use torchrun.
Note that --use_env is set by default in torchrun.
If your script expects `--local_rank` argument to be set, please
change it to read from `os.environ['LOCAL_RANK']` instead. See 
https://pytorch.org/docs/stable/distributed.html#launch-utility for 
further instructions

  FutureWarning,
Training with a single process on 1 GPUs.
Loading pretrained weights from url (https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-rsb-weights/resnet50_a1_0-14fe96d1.pth)
Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-rsb-weights/resnet50_a1_0-14fe96d1.pth" to /root/.cache/torch/hub/checkpoints/resnet50_a1_0-14fe96d1.pth
Model resnet50 created, param count:24122732
Data processing configuration for current model + dataset:
	input_size: (3, 600, 600)
	interpolation: